### 1. API로 데이터 수집

In [3]:
import requests
import xmltodict
import json
import numpy as np
import pandas as pd

In [4]:
url = 'http://openapi.gbis.go.kr/ws/rest/baseinfoservice'

queryParams = '?' + 'serviceKey' + '=' + 'uNY%2BOVwL3weHrcP9MP3EjeBGgmlirGAMsaVwv%2FDRvG4i6h7StXaoWWHqTTS5g0T%2FFIf7HyW2GZCwL2%2F6lQvsvA%3D%3D'
# queryParams += '&' + 'routeNo' + '=' + '5'
response = requests.get(url + queryParams)

if(response.status_code == 200):
    responseData = response.text #요청받은 XML 데이터 저장
    rD = xmltodict.parse(responseData) #XML -> dict형식으로 변환
    rDJ = json.dumps(rD) #dict -> json
    rDD = json.loads(rDJ) #json-=> dict
else:
    print("Error Code:" , rescode)


In [5]:
rDD['response']['msgBody']['baseInfoItem']

{'areaDownloadUrl': 'http://openapi.gbis.go.kr/ws/download?area20200223.txt',
 'areaVersion': '20200223',
 'routeDownloadUrl': 'http://openapi.gbis.go.kr/ws/download?route20200223.txt',
 'routeLineDownloadUrl': 'http://openapi.gbis.go.kr/ws/download?routeline20200223.txt',
 'routeLineVersion': '20200223',
 'routeStationDownloadUrl': 'http://openapi.gbis.go.kr/ws/download?routestation20200223.txt',
 'routeStationVersion': '20200223',
 'routeVersion': '20200223',
 'stationDownloadUrl': 'http://openapi.gbis.go.kr/ws/download?station20200223.txt',
 'stationVersion': '20200223'}

### 2. 필요한 정보만 데이터 프레임으로 만들기

In [6]:
temp = np.loadtxt('경기버스기반정보/route20200222.txt', dtype='object', delimiter='^', encoding='utf-8')

route = pd.DataFrame(columns=temp[0].split('|'))

for i,col in enumerate(route.columns):
    route[col] = pd.DataFrame(temp[1:])[0].str.split('|').str[i]

# 기점, 종점 정류소명
route = route[['ROUTE_ID','ROUTE_NM','ROUTE_TP','ST_STA_ID','ST_STA_NM','ST_STA_NO','ED_STA_ID','ED_STA_NM','ED_STA_NO','REGION_NAME','DISTRICT_CD']]
route.sort_values(by=['ROUTE_NM','ROUTE_ID'], ascending=True, inplace=True)
route.reset_index(drop=True, inplace=True)
route.head()

,ROUTE_ID,ROUTE_NM,ROUTE_TP,ST_STA_ID,ST_STA_NM,ST_STA_NO,ED_STA_ID,ED_STA_NM,ED_STA_NO,REGION_NAME,DISTRICT_CD
0,207000016,1,13,207000616,현대힐스테이트아파트,08582,222000933,청학주공5단지,23027,"남양주,의정부",2
1,208000007,1,13,226000206,숲속마을3.5단지,27199,116000372,구로디지털단지역환승센터(2번승강장),,"서울,안양,의왕",2
2,213000006,1,13,224000449,거모동,25305,116000609,개봉역,85187,"광명,서울,시흥",2
3,215000025,1,13,215000003,구)버스터미널,16283,215000091,안말회관,16118,동두천,2
4,222000058,1,13,222001119,진벌리차고지,49204,104000183,강변역(A),,"구리,남양주,서울",2


In [7]:
temp = np.loadtxt('경기버스기반정보/routestation20200222.txt', dtype='object', delimiter='^', encoding='utf-8')

routestation = pd.DataFrame(columns=temp[0].split('|'))

for i,col in enumerate(routestation.columns):
    routestation[col] = pd.DataFrame(temp[1:])[0].str.split('|').str[i]

# 노선별 정류장 정보
routestation['STA_ORDER'] = routestation['STA_ORDER'].astype('float64')
routestation.sort_values(by=['ROUTE_NM','ROUTE_ID','STA_ORDER'], ascending=True, inplace=True)
routestation.reset_index(drop=True, inplace=True)
routestation.head() 

,ROUTE_ID,STATION_ID,UPDOWN,STA_ORDER,ROUTE_NM,STATION_NM
0,207000016,207000616,정,1.0,1,현대힐스테이트아파트
1,207000016,207000546,정,2.0,1,의정부법원.검찰청입구
2,207000016,207000276,정,3.0,1,미2사단앞
3,207000016,207000275,정,4.0,1,경민대학
4,207000016,207000274,정,5.0,1,안골


### 3. DB 구축

In [27]:
import os
import cx_Oracle

In [38]:
# 연결
os.putenv('NLS_LANG','KOREAN_KOREA.KO16MSWIN949')
con = cx_Oracle.connect("hr/1234@localhost:1521/xe")
cursor = con.cursor()

# 테이블 생성
cursor.execute("create table route (ROUTE_ID varchar2(15),ROUTE_NM varchar2(20),ROUTE_TP varchar2(3),ST_STA_ID varchar2(15),ST_STA_NM varchar2(150),ST_STA_NO varchar2(15),ED_STA_ID varchar2(15),ED_STA_NM varchar2(150),ED_STA_NO varchar2(15),REGION_NAME varchar2(150),DISTRICT_CD varchar2(1))")
cursor.execute("create table routestation (ROUTE_ID varchar2(15),STATION_ID varchar2(15),UPDOWN varchar2(5),STA_ORDER varchar2(5),ROUTE_NM varchar2(20),STATION_NM varchar2(150))")

# insert data
rows_route = [tuple(x) for x in route.to_records(index=False)]
rows_routestation = [tuple(x) for x in routestation.to_records(index=False)]
cursor.executemany('insert into route values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)', rows_route)
cursor.executemany('insert into routestation values (:1, :2, :3, :4, :5, :6)', rows_routestation)

# commit
con.commit()

# 연결 종료
con.close()